In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
# import logging
# logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]" #これは自分で作らなきゃいけないのかな
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [2]:
#自分でやってみる

# Tokenized input
text = r"I am a student." 


#単語とか活用とかそれらをいい感じに区切ってくれるやつ
tokenized_text = tokenizer.tokenize(text)
# [CLF]を先頭につけなければいけない、最後には[SEP]をつけなければ行けないが
tokenized_text= ['[CLS]']+tokenized_text+['[SEP]']
print(tokenized_text)

# 単語番号に変換するためのコンバーター
# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)

# Convert inputs to PyTorch tensors
# テンソル化
tokens_tensor = torch.tensor([indexed_tokens])

#文章が一つの場合にもセグメントテンサーが必要なのかも知れない
segments_tensors=torch.tensor([0]*len(indexed_tokens))

['[CLS]', 'i', 'am', 'a', 'student', '.', '[SEP]']
[101, 1045, 2572, 1037, 3076, 1012, 102]


Let's see how to use `BertModel` to get hidden states

In [3]:
# Load pre-trained model (weights)
# BERTのモデルを持ってくる
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cpu')
segments_tensors = segments_tensors.to('cpu')
model.to('cpu')

# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
# We have a hidden states for each of the 12 layers in model bert-base-uncased

In [4]:
len(encoded_layers)

12

In [5]:
encoded_layers[-1].shape

torch.Size([1, 7, 768])

encoded_layersの中身はもしかして、(bertの層(リスト), トークン数, hidden state) になっている？

In [6]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-large-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cpu')
segments_tensors = segments_tensors.to('cpu')
model.to('cpu')

# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
# We have a hidden states for each of the 12 layers in model bert-base-uncased

In [7]:
encoded_layers

[tensor([[[-3.5949e-02, -1.9778e-02, -1.1472e-01,  ...,  2.4263e-04,
           -1.1829e-01, -3.2291e-03],
          [-3.0887e-01, -3.6599e-01,  3.7903e-01,  ...,  6.8173e-01,
            6.6305e-01,  5.7927e-01],
          [-1.0917e+00,  5.2833e-01, -3.2227e-01,  ...,  1.9574e-01,
            2.2541e+00,  1.1525e-01],
          ...,
          [-4.7768e-01,  6.3005e-01,  4.5194e-02,  ...,  9.5886e-01,
            1.0072e+00, -1.2784e+00],
          [ 3.3015e-01, -6.7547e-03, -7.5185e-03,  ...,  5.8924e-01,
            4.2048e-01,  6.7004e-02],
          [ 6.3963e-02,  3.4640e-01, -2.2910e-02,  ..., -1.0631e-01,
            1.4546e-01, -8.2015e-02]]]),
 tensor([[[-0.0285,  0.0497, -0.0365,  ...,  0.0043, -0.0979, -0.0424],
          [-0.4379, -0.2310,  0.3101,  ...,  0.3559,  0.3849,  0.6761],
          [-0.9673,  0.2351, -0.1300,  ..., -0.1953,  2.2468,  0.3808],
          ...,
          [-0.6117,  0.8534,  0.1567,  ...,  0.6797,  0.7153, -1.0427],
          [ 0.1356,  0.0770,  0.2520,

In [8]:
# 12レイヤー*

encoded_layers[-1].shape

torch.Size([1, 7, 1024])

In [9]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cpu')
segments_tensors = segments_tensors.to('cpu')
model.to('cpu')

# Predict hidden states features for each layer
with torch.no_grad():
    last_layers, _ = model(tokens_tensor, segments_tensors,output_all_encoded_layers=False)
    #最終層だけ取り出す。
# We have a hidden states for each of the 12 layers in model bert-base-uncased
# assert len(encoded_layers) == 1

In [11]:
encoded_layers[11].shape

torch.Size([1, 7, 1024])

In [12]:
encoded_layers

[tensor([[[-3.5949e-02, -1.9778e-02, -1.1472e-01,  ...,  2.4263e-04,
           -1.1829e-01, -3.2291e-03],
          [-3.0887e-01, -3.6599e-01,  3.7903e-01,  ...,  6.8173e-01,
            6.6305e-01,  5.7927e-01],
          [-1.0917e+00,  5.2833e-01, -3.2227e-01,  ...,  1.9574e-01,
            2.2541e+00,  1.1525e-01],
          ...,
          [-4.7768e-01,  6.3005e-01,  4.5194e-02,  ...,  9.5886e-01,
            1.0072e+00, -1.2784e+00],
          [ 3.3015e-01, -6.7547e-03, -7.5185e-03,  ...,  5.8924e-01,
            4.2048e-01,  6.7004e-02],
          [ 6.3963e-02,  3.4640e-01, -2.2910e-02,  ..., -1.0631e-01,
            1.4546e-01, -8.2015e-02]]]),
 tensor([[[-0.0285,  0.0497, -0.0365,  ...,  0.0043, -0.0979, -0.0424],
          [-0.4379, -0.2310,  0.3101,  ...,  0.3559,  0.3849,  0.6761],
          [-0.9673,  0.2351, -0.1300,  ..., -0.1953,  2.2468,  0.3808],
          ...,
          [-0.6117,  0.8534,  0.1567,  ...,  0.6797,  0.7153, -1.0427],
          [ 0.1356,  0.0770,  0.2520,

いろいろな文の内部表現を取り出したい

In [13]:
text = "[CLS] I like unko! [SEP] can I input several sentencies? [SEP] ok? [SEP]"
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])

In [14]:
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor)


In [15]:
encoded_layers[-1].shape

torch.Size([1, 7, 768])